In [1]:
import nltk
import spacy
import pandas as pd
import re

In [2]:
import json
with open('tweets.json') as file:
    df = json.load(file)

In [3]:
tweets = []
authors = []
for key in df:
    tweets.append(df[key]['tweet_text'])
    authors.append(df[key]['tweet_author'])

In [4]:
authors[:5]

['Hematopoiesis News',
 'Michael Wang, MD',
 '1stOncology',
 'Toby Eyre',
 'Lymphoma Hub']

In [5]:
data = {'Author': authors, 'Tweet': tweets}
df = pd.DataFrame(data)

In [6]:
df["cleaned_tweets"] = [re.sub(r'http\S+', '', tweet) for tweet in df['Tweet']]

df["cleaned_tweets"][0]

'⚕️ Scientists conducted a Phase II study of acalabrutinib in patients with relapsed/refractory #CLL who were ibrutinib-intolerant, and found an overall response rate of 73%. \n '

In [7]:
df["cleaned_tweets"] = [re.sub(r'#\S+', '', tweet) for tweet in df["cleaned_tweets"]]

df["cleaned_tweets"][0]

'⚕️ Scientists conducted a Phase II study of acalabrutinib in patients with relapsed/refractory  who were ibrutinib-intolerant, and found an overall response rate of 73%. \n '

In [8]:
df["cleaned_tweets"] = [re.sub(r'[^A-Za-z0-9\s]+', '', tweet) for tweet in df["cleaned_tweets"]]

df["cleaned_tweets"][0]

' Scientists conducted a Phase II study of acalabrutinib in patients with relapsedrefractory  who were ibrutinibintolerant and found an overall response rate of 73 \n '

In [9]:
df["cleaned_tweets"] = [re.sub(r'\n', '', tweet) for tweet in df["cleaned_tweets"]]

df["cleaned_tweets"][0]

' Scientists conducted a Phase II study of acalabrutinib in patients with relapsedrefractory  who were ibrutinibintolerant and found an overall response rate of 73  '

In [10]:
df["cleaned_tweets"] = [re.sub(r'\d+', '', tweet) for tweet in df["cleaned_tweets"]]

df["cleaned_tweets"][1]

'This phase  AcalabrutinibVenetoclax AV trial that is still in recruitment phase will study how well venetoclax and acalabrutinib works in MCL patients who either relapsed or nonrespondent to the initial therapy'

In [11]:
nlp = spacy.load("en_core_web_sm")

def text_preprocessing(text):
    doc = nlp(text)
    removing_stopword =  [token for token in doc if not token.is_stop]
    lemmatization = [token.lemma_ for token in removing_stopword]
    preprocessed_tweets = " ".join(lemmatization)
    return preprocessed_tweets

In [12]:
df["cleaned_tweets"] = df["cleaned_tweets"].apply(text_preprocessing)

In [13]:
def extracting_entities(text):
    doc = nlp(text)
    entities = []
    for ent in doc.ents:
        entities.append(ent.text)
               
    return entities

In [14]:
df['entities'] = df['cleaned_tweets'].apply(extracting_entities)

In [15]:
df['entities'] = df['entities'].apply(lambda x: ' '.join(x))

In [19]:
def get_polarity(text):
    doc = nlp(text)
    polarity = 0
    for token in doc:
        polarity += token.sentiment
    
    if polarity > 0:
        return "positive"
    else:
        return "negative"

In [20]:
df['combined_tweets'] = df['Author'] + " " + df['entities']

In [21]:
df['overall_polarity'] = df['combined_tweets'].apply(get_polarity)

In [17]:
df.head()

Author                                              Tweet  \
0  Hematopoiesis News  ⚕️ Scientists conducted a Phase II study of ac...   
1    Michael Wang, MD  This phase 2 Acalabrutinib-Venetoclax (AV) tri...   
2         1stOncology  #NICE backs #AstraZenecas #Calquence for #CLL ...   
3           Toby Eyre  #acalabrutinib is a valuable option in pts int...   
4        Lymphoma Hub  NICE has recommended the use of acalabrutinib ...   

                                      cleaned_tweets  \
0    scientist conduct Phase II study acalabrutin...   
1  phase   AcalabrutinibVenetoclax AV trial recru...   
2                                          back        
3    valuable option pt intolerant   valuable dat...   
4  NICE recommend use acalabrutinib patient treat...   

                         entities overall_polarity  
0                        Phase II         negative  
1  AcalabrutinibVenetoclax AV MCL         negative  
2                                         negative  
3                                         negative  
4                                         negative

In [24]:
columns = ['entities','Author','overall_polarity']

In [25]:
new_df = df[columns]

In [26]:
new_df.head()

entities              Author overall_polarity
0                        Phase II  Hematopoiesis News         negative
1  AcalabrutinibVenetoclax AV MCL    Michael Wang, MD         negative
2                                         1stOncology         negative
3                                           Toby Eyre         negative
4                                        Lymphoma Hub         negative